## LLMを使用してテキストから情報を抽出する

LLMは言語を「理解」するため、感情分析や情報抽出のようなタスクに適しています。

このノートブックでは、クレームを分析して、書いた人の感情状態や事故の場所、時間を特定するためにLLMを使用します。

### 必要条件とインポート

ラボの指示に従って正しいワークベンチイメージを選択した場合、必要なライブラリはすでにインストールされています。そうでない場合は、次のセルの最初の行のコメントを外して、必要なパッケージをインストールしてください。

In [ ]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # 正しいワークベンチイメージを選択していない場合のみ、コメントを外してください

import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain_community.llms import VLLMOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

### Langchainパイプライン

再び、Langchainを使用してタスクパイプラインを定義します。

In [ ]:
# LLM推論サーバーのURL
inference_server_url = "http://llm.ic-shared-llm.svc.cluster.local:8000"

# LLMの定義
llm = VLLMOpenAI(           # vLLMのOpenAI互換APIクライアントを使用していますが、モデルはOpenShift上で実行されています。
    openai_api_key="EMPTY",   # OpenAIのキーは必要ありません。
    openai_api_base= f"{inference_server_url}/v1",
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

次に、さまざまなタスクに使用する**テンプレート**を定義します。

In [ ]:
template="""<s>[INST]
あなたは役立ち、尊敬を持ち、誠実なアシスタントです。
常に注意深く、尊敬を持って、真実をもって支援してください。最大限に役立ちつつも、安全な方法で返信してください。
有害、非倫理的、偏見に満ちた、または否定的な内容を避け、公平さと前向きさを促進する返信を心がけてください。
次に示すテキストに基づいて質問に答えてください。できるだけ正確で簡潔な回答を提供してください。

### テキスト:
{text}

### 質問:
{query}

### 回答:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["text", "query"], template=template)

次に、モデルにクエリを送信するために使用する**会話**オブジェクトを作成します。

In [ ]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )

これでモデルにクエリを送信する準備が整いました！

`claims`フォルダには、受け取ることができるクレームの例が含まれたJSONファイルがあります。それらのファイルを読み込み、表示し、その後LLMが行った分析を確認します。

In [ ]:
# クレームを読み込み、辞書に追加

claims_path = 'claims'
onlyfiles = [f for f in listdir(claims_path) if isfile(join(claims_path, f))]

claims = {}

for filename in onlyfiles:
    # JSONファイルを開く
    with open(os.path.join(claims_path, filename), 'r') as file:
        data = json.load(file)
    claims[filename] = data

In [ ]:
# クレームを分析

for filename in onlyfiles:
    print(f"***************************")
    print(f"* クレーム: {filename}")
    print(f"***************************")
    print("オリジナルコンテンツ:")
    print("-----------------")
    print(f"件名: {claims[filename]['subject']}\nコンテンツ:\n{claims[filename]['content']}\n\n")
    print('分析結果:')
    print("--------")
    text_input = f"件名: {claims[filename]['subject']}\nコンテンツ:\n{claims[filename]['content']}"
    sentiment_query = "このクレームを送信した人の感情はどうですか？"
    location_query = "クレームに関連する出来事が発生した場所はどこですか？"
    time_query = "クレームに関連する出来事が発生した日時はいつですか？可能であれば、具体的な日付と時間を教えてください。"
    print(f"- 感情: ")
    conversation.predict(text=text_input, query=sentiment_query);
    print("\n- 場所: ")
    conversation.predict(text=text_input, query=location_query);
    print("\n- 日時: ")
    conversation.predict(text=text_input, query=time_query);
    print("\n\n                          ----====----\n")

必要に応じて、セクション3.7に戻って追加の演習を試してみてください。